<a href="https://colab.research.google.com/github/TK-Problem/Coursera-scrapper/blob/master/coursera_review_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Load libraries
import requests # for running HTML requests
from tqdm import tqdm # for monitoring progress
from bs4 import BeautifulSoup # for parsing raw HTML code and extracting usefull data
import pandas as pd # for reading csv file

![example](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/colab-upload.png?ssl=1 "Placeholder fig. title")

or one can load from google drive (faster option)

![example](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/colab-file.png?ssl=1 "Placeholder fig. title")



In [ ]:
#@title Read URLs

#@markdown Upload data before running this cell upload
#@markdown `.csv` files manually into enviroment (see image above).

file_path = '/content/drive/MyDrive/courses_detailed.csv'  #@param {type: "string"}

# read data
try:
  df = pd.read_csv(file_path, header=None)
  # select unique URLS
  urls = df.iloc[:, 0].unique()

# file path/fole not found
except FileNotFoundError:
  print("File not found.")
  urls = []

# return status message
print(f"{len(urls)} courses found")